In [4]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras import optimizers
import argparse
from RNN_utils import *

DATA_DIR = './data/shakespeare_input.txt'
BATCH_SIZE = 30
HIDDEN_DIM = 300
SEQ_LENGTH = 50

lossfile = './data/warpeace_loss.txt'

GENERATE_LENGTH = 100
LAYER_NUM = 2

# Creating training data
X, y, VOCAB_SIZE, ix_to_char, char_to_ix = load_data(DATA_DIR, SEQ_LENGTH)

# Creating and compiling the Network
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
  model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
rms_prop = optimizers.RMSprop(lr=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=rms_prop)

# Generate some sample before training to know how bad it is!
# print("\n\nGenerating untrained text...\n\n")
# generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char, -1, 0)



Data length: 4573338 characters
Vocabulary size: 67 characters


In [7]:
import sys
import numpy as np
import tensorflow as tf
from datetime import datetime

device_name="/gpu:0"

shape=(int(10000),int(10000))

with tf.device(device_name):
    random_matrix = tf.random_uniform(shape=shape, minval=0, maxval=1)
    dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
    sum_operation = tf.reduce_sum(dot_operation)

startTime = datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
        result = session.run(sum_operation)
        print(result)

print("\n" * 2)
print("Shape:", shape, "Device:", device_name)
print("Time taken:", datetime.now() - startTime)

print("\n" * 2)


250008720000.0



Shape: (10000, 10000) Device: /gpu:0
Time taken: 0:00:01.927762





In [8]:
import keras.callbacks
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

In [9]:
history = model.fit(X, y, batch_size=BATCH_SIZE, validation_split=0.2, epochs=2, verbose=1)

Train on 73172 samples, validate on 18294 samples
Epoch 1/2
69300/73172 [===========================>..] - ETA: 1:20 - loss: 2.9948

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
history